# Performs Frequent Itemset Mining on different relations defined in 'related_to' feature of the product dataset

## Imports

In [9]:
import sys
project_path = "/media/sarthak/HDD/TUM/courses/sem 3/practical DM/Amazon-Mining"
project_path = "D:\\TUM\\courses\\sem 3\\practical DM\\Amazon-Mining"
# make sure to use position 1
sys.path.insert(1, project_path)

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
import itertools
import pandas as pd
import _pickle as cPickle
from pandas import DataFrame
from src.model import apriori
from pandas import Series

# Function definitions

In [11]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [12]:
def get_cat(l):
    return l[1]

## These 4 different functions are basically copies with different 'related_to' relation. These can be merged into a single function

In [13]:
def find_cat_ab(col):
    categories_without_none = []
    if not pd.isnull(col[7]):
        asins = col[7].get('also_bought')
        if asins:
            categories = [asin_cat_dict.get(asin) for asin in asins]
            categories_without_none = [x for x in categories if x is not None]
            categories_without_none.append(asin_cat_dict.get(col[0]))
    return categories_without_none

In [14]:
def find_cat_bt(col):
    categories_without_none = []
    if not pd.isnull(col[7]):
        asins = col[7].get('bought_together')
        if asins:
            categories = [asin_cat_dict.get(asin) for asin in asins]
            categories_without_none = [x for x in categories if x is not None]
            categories_without_none.append(asin_cat_dict.get(col[0]))
    return categories_without_none

In [15]:
def find_cat_av(col):
    categories_without_none = []
    if not pd.isnull(col[7]):
        asins = col[7].get('also_viewed')
        if asins:
            categories = [asin_cat_dict.get(asin) for asin in asins]
            categories_without_none = [x for x in categories if x is not None]
            categories_without_none.append(asin_cat_dict.get(col[0]))
    return categories_without_none

In [16]:
def find_cat_bav(col):
    categories_without_none = []
    if not pd.isnull(col[7]):
        asins = col[7].get('buy_after_viewing')
        if asins:
            categories = [asin_cat_dict.get(asin) for asin in asins]
            categories_without_none = [x for x in categories if x is not None]
            categories_without_none.append(asin_cat_dict.get(col[0]))
    return categories_without_none

# data loading

In [19]:
from src.data.json_loader import JSONLoader
# file_path corresponds to the file of the .gz file which contains the JSON file. 
# product_path = '/media/sarthak/HDD/TUM/courses/sem 3/practical DM/datasets/meta_Electronics.json.gz'
product_path = 'D:\\TUM\\courses\\sem 3\\practical DM\\datasets\\meta_Electronics.json.gz'
loader = JSONLoader()
product = loader.load_data(product_path)

In [20]:
product.categories = product.categories.apply(flatten)
product.categories = product.categories.apply(get_cat)

# code that did the magic of running in sub-seconds complexity.
asin_cat_dict = Series(product.categories.values,index=product.asin).to_dict()

In [21]:
product.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,Computers & Accessories,Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,Computers & Accessories,Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,Computers & Accessories,Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,Accessories & Supplies,CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...",GPS & Navigation,Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


# getting categories for relations

In [22]:
categories_also_bought = product.apply(find_cat_ab, axis=1)
categories_bought_together = product.apply(find_cat_bt, axis=1)
categories_also_viewed = product.apply(find_cat_av, axis=1)
categories_buy_after_viewing = product.apply(find_cat_bav, axis=1)

In [23]:
categories_for_aprioiri_also_bought = [x for x in categories_also_bought if len(x) is not 0]
categories_for_aprioiri_bought_together = [x for x in categories_bought_together if len(x) is not 0]
categories_for_aprioiri_also_viewed = [x for x in categories_also_viewed if len(x) is not 0]
categories_for_aprioiri_buy_after_viewing = [x for x in categories_buy_after_viewing if len(x) is not 0]

# Frequent Itemset Mining

In [24]:
L_AB, suppData_AB = apriori.apriori(categories_for_aprioiri_also_bought, minSupport=0.10)
L_BT, suppData_BT = apriori.apriori(categories_for_aprioiri_bought_together, minSupport=0.10)
L_AV, suppData_AV = apriori.apriori(categories_for_aprioiri_also_viewed, minSupport=0.10)
L_BAV, suppData_BAV = apriori.apriori(categories_for_aprioiri_buy_after_viewing, minSupport=0.10)

In [29]:
L_AB

[[frozenset({'Electronics Warranties'}),
  frozenset({'Portable Audio & Video'}),
  frozenset({'Camera & Photo'}),
  frozenset({'Car & Vehicle Electronics'}),
  frozenset({'Accessories & Supplies'}),
  frozenset({'Computers & Accessories'})],
 [frozenset({'Camera & Photo', 'Computers & Accessories'}),
  frozenset({'Accessories & Supplies', 'Camera & Photo'}),
  frozenset({'Accessories & Supplies', 'Computers & Accessories'})],
 []]

In [30]:
L_BT

[[frozenset({'Camera & Photo'}),
  frozenset({'Accessories & Supplies'}),
  frozenset({'Computers & Accessories'})],
 []]

In [31]:
L_AV

[[frozenset({'Camera & Photo'}),
  frozenset({'Car & Vehicle Electronics'}),
  frozenset({'Portable Audio & Video'}),
  frozenset({'Accessories & Supplies'}),
  frozenset({'Computers & Accessories'})],
 [frozenset({'Accessories & Supplies', 'Computers & Accessories'})],
 []]

In [32]:
L_BAV

[[frozenset({'Camera & Photo'}),
  frozenset({'Computers & Accessories'}),
  frozenset({'Accessories & Supplies'})],
 []]

In [25]:
suppData_AB

{frozenset({'Computers & Accessories'}): 0.693320637607561,
 frozenset({'eBook Readers & Accessories'}): 0.026350684158555508,
 frozenset({'Accessories & Supplies'}): 0.42436874030187616,
 frozenset({'Car & Vehicle Electronics'}): 0.12714769360981804,
 frozenset({'Camera & Photo'}): 0.2947453801664551,
 frozenset({'Home Audio'}): 0.06384539427281705,
 frozenset({'Portable Audio & Video'}): 0.1351107349414586,
 frozenset({'Television & Video'}): 0.07002398081534772,
 frozenset({'Cases'}): 0.015968401749188885,
 frozenset({'Electronics Warranties'}): 0.10308224009028072,
 frozenset({'Accessories'}): 0.05664409648751587,
 frozenset({'Security & Surveillance'}): 0.04503456058682466,
 frozenset({'GPS & Navigation'}): 0.042431936803498375,
 frozenset({'Connected Devices'}): 0.0005430949358160531,
 frozenset({'Service & Replacement Plans'}): 0.005818874312314854,
 frozenset({'Interior Accessories'}): 5.642544787699252e-05,
 frozenset({'Car Care'}): 0.0002539145154464664,
 frozenset({'Tools & 

In [26]:
suppData_BT

{frozenset({'Computers & Accessories'}): 0.4805786649251824,
 frozenset({'eBook Readers & Accessories'}): 0.010739941508844658,
 frozenset({'Car & Vehicle Electronics'}): 0.07260285641273176,
 frozenset({'Accessories & Supplies'}): 0.19857179363411795,
 frozenset({'Camera & Photo'}): 0.23220420795593288,
 frozenset({'Electronics Warranties'}): 0.025107186461852976,
 frozenset({'Television & Video'}): 0.016269627189869105,
 frozenset({'Portable Audio & Video'}): 0.06184161958034016,
 frozenset({'Security & Surveillance'}): 0.01526164854198018,
 frozenset({'Accessories'}): 0.009717765978591102,
 frozenset({'GPS & Navigation'}): 0.018569522132939605,
 frozenset({'Home Audio'}): 0.01918708651580113,
 frozenset({'Cases'}): 0.0051676651807263125,
 frozenset({'Service & Replacement Plans'}): 0.0007666316476901673,
 frozenset({'Car Care'}): 0.00014906726482864362,
 frozenset({'Interior Accessories'}): 7.098441182316364e-06,
 frozenset({'Tools & Equipment'}): 3.549220591158182e-05,
 frozenset({

In [27]:
suppData_AV

{frozenset({'Computers & Accessories'}): 0.5850489360027289,
 frozenset({'Accessories & Supplies'}): 0.24882579832594265,
 frozenset({'Portable Audio & Video'}): 0.11495368791162656,
 frozenset({'Television & Video'}): 0.06785442523156045,
 frozenset({'Car & Vehicle Electronics'}): 0.12057411246096927,
 frozenset({'GPS & Navigation'}): 0.026889874314502373,
 frozenset({'eBook Readers & Accessories'}): 0.01974233161029624,
 frozenset({'Camera & Photo'}): 0.222271785048936,
 frozenset({'Security & Surveillance'}): 0.026013486919786936,
 frozenset({'Accessories'}): 0.0278764661121461,
 frozenset({'Cases'}): 0.010490409593030884,
 frozenset({'Cell Phones'}): 6.297394453045052e-05,
 frozenset({'Home Audio'}): 0.07168534019049619,
 frozenset({'Interior Accessories'}): 0.0001889218335913516,
 frozenset({'Pop'}): 5.247828710870877e-06,
 frozenset({'Tools & Equipment'}): 9.970874550654666e-05,
 frozenset({'RV Parts & Accessories'}): 0.00012594788906090105,
 frozenset({'Electronics Warranties'})

In [28]:
suppData_BAV

{frozenset({'Accessories & Supplies'}): 0.20679055074506644,
 frozenset({'GPS & Navigation'}): 0.017368103101087395,
 frozenset({'Computers & Accessories'}): 0.5304696939186468,
 frozenset({'eBook Readers & Accessories'}): 0.016141008860249696,
 frozenset({'Car & Vehicle Electronics'}): 0.06525938884414015,
 frozenset({'Camera & Photo'}): 0.2091377617801047,
 frozenset({'Portable Audio & Video'}): 0.08022679218687072,
 frozenset({'Cases'}): 0.005068843133306484,
 frozenset({'Television & Video'}): 0.02850948952879581,
 frozenset({'Security & Surveillance'}): 0.015901882803060815,
 frozenset({'Accessories'}): 0.013469719089810712,
 frozenset({'Home Audio'}): 0.027540399718082965,
 frozenset({'Interior Accessories'}): 4.090314136125655e-05,
 frozenset({'Electronics Warranties'}): 0.009115107732581555,
 frozenset({'Replacement Parts'}): 2.5171163914619413e-05,
 frozenset({'Service & Replacement Plans'}): 0.0005317408376963351,
 frozenset({'Connected Devices'}): 0.00014788058799838905,
 fr